# Compute Scaling Study 

## Objectives:

We measure the performance of Dask’s distributed scheduler for a variety of different operations commonly performed in geosciences (`climatology`, `anomaly`, `temporal` and `global` reductions). We measure performance under increasing scales of both dataset size and cluster size.


During this study, we vary our computations in following ways:

- Varying chunk size
- Varying cluster size (workers per node, number of nodes)
- Varying chunking scheme
- Varying machines (cheyene, hal)_

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import dask.dataframe as dd
import hvplot.pandas
from distributed.utils import format_bytes, parse_bytes

In [ ]:
df = dd.read_csv('results*/*.csv').compute()
df['chunk_size'] = df['chunk_size'].map(lambda x: format_bytes(parse_bytes(x)))

In [ ]:
df.head()

In [ ]:
len(df)

## Weak Scaling


[Weak scaling](https://en.wikipedia.org/wiki/Scalability#Weak_versus_strong_scaling) is how the time to solution varies with processor count with a fixed system size per processor. 


In an ideal case (e.g., problems/algorithms with O(N) time complexity), we expect to observe a constant time to solution, independent of the total number of processors in the system. 


In [ ]:
def get_clean_df(df,
                 groupby=[
                     'chunk_size', 'chunking_scheme', 'dataset_size',
                     'operation', 'worker_per_node', 'num_nodes', 'machine'
                 ]):
    clean_df = df.groupby(groupby).median().reset_index()
    return clean_df

In [ ]:
df1 = get_clean_df(df,
                   groupby=[
                       'chunk_size', 'chunking_scheme', 'operation',
                       'worker_per_node', 'num_nodes', 'machine'
                   ])
df1.head()

In [ ]:
temp_df = df.copy()
temp_df['cores'] = temp_df['num_nodes'] * temp_df['worker_per_node']
temp_df.head()

In [ ]:
df2 = get_clean_df(temp_df, groupby=['chunk_size', 'chunking_scheme', 'operation', 'cores', 'machine'])\
    .drop(columns=['num_nodes', 'worker_per_node'])
df2.head()

In [ ]:
def log_linear_plot(
        df,
        loglog=True,
        plot_kind='line',
        x='num_nodes',
        y='runtime',
        by='chunk_size',
        groupby=['chunking_scheme', 'operation', 'worker_per_node',
                 'machine']):
    if loglog:
        title = f'{y} vs {x} -- Log scale'
    else:
        title = f'{y} vs {x} -- Linear scale'
    p = df.hvplot(x=x,
                  y=y,
                  by=by,
                  groupby=groupby,
                  rot=45,
                  loglog=loglog,
                  height=500,
                  kind=plot_kind,
                  title=title,
                  ylabel='Runtime (seconds)',
                  dynamic=False,
                  legend='top')
    return p

In [ ]:
log_linear_plot(df1)

In [ ]:
log_linear_plot(df1, loglog=False)

In [ ]:
log_linear_plot(df2,
                x='cores',
                groupby=['chunking_scheme', 'operation', 'machine'])

In [ ]:
log_linear_plot(df2,
                loglog=False,
                x='cores',
                groupby=['chunking_scheme', 'operation', 'machine'])

## Strong Scaling

In [ ]:
df3 = get_clean_df(temp_df,
                   groupby=[
                       'dataset_size', 'cores', 'operation', 'chunking_scheme',
                       'machine'
                   ])
df3.head()

In [ ]:
log_linear_plot(df3,
                x='cores',
                y='runtime',
                by='dataset_size',
                groupby=['operation', 'chunking_scheme', 'machine'])

In [ ]:
log_linear_plot(df3,
                x='cores',
                y='runtime',
                by='dataset_size',
                groupby=['operation', 'chunking_scheme', 'machine'],
                loglog=False)